In [1]:
%pip install -q -U google-generativeai

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: colab 1.13.5 has a non-standard dependency specifier pytz>=2011n. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of colab or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [2]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


In [3]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: colab 1.13.5 has a non-standard dependency specifier pytz>=2011n. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of colab or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [4]:
from dotenv import dotenv_values

# Load environment variables from .env file
env_vars = dotenv_values(".env")

GOOGLE_API_KEY = env_vars.get("API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)


In [5]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-pro
models/gemini-pro-vision


In [6]:
model = genai.GenerativeModel('gemini-1.0-pro')

In [7]:
description = "assume that the review is about a software application and classify the review based on multiple labels: 'F':'Feature Request','AU':'Authentication','FI':'Filtering', 'A':'Availability', 'L':'Legal', 'LF':'Look-and-Feel', 'MN':'Maintainability', 'O':'Operability', 'PE':'Performance', 'SC':'Scalability', 'SE':'Security', 'US':'Usability', 'BR':'Bug Report', 'PO':'Portability', 'IR':'Irrelevant'. Positive reviews/compliments or sensitive content or inappropriate context or irrelevant comments or statements that are incomprehensible or that does not specify any issue/requirement are considered as IR, and cannot be classified under any other label. Issues associated with bugs/crashes/errors in the application should be classified as BR. Only issues relating to login/authentication errors, authorization, OTP, or sign-in should fall under AU. Any issues pertaining to functionality such as searching, filtering, sorting, arranging, organizing, or grouping can fall under FI. Issues related to the unavailability of the application in certain countries or on certain devices may fall under A. Legal issues, subscription matters, in-app purchases, or terms and conditions concerns can come under L. Any aspect not classified as a functional requirement and related to the visual and feel of the app may come under LF. Issues concerning customer service, inadequate support, inconsistent updates, or limited functionality will be classified under MN. Any problems related to abnormal functioning of the app or issues in opening, updating, or working of the app may fall into O. Issues related to limited quantity, version scalability, or expansion of a particular feature may come under SC. Any concerns regarding privacy and security of the user will come under SE. Personal concerns or issues in using the application may fall under US. Issues pertaining to the compatibility of the application fall under PO. The response must be a list of the abbreviations of the class the review falls under from - [F, BR, AU, FI, IR, A, L, LF, MN, O, PE, SC, SE, US, PO] (they must be comma seperated, not in bulletin points).Strictly no other text should be delivered as response just the list!, also give explanation on the next line."

In [8]:
classes = ["F", "BR", "AU", "FI", "IR", "A", "L", "LF", "MN", "O", "PE", "SC", "SE", "US", "PO"]

In [9]:
async def Classify(review):
    prompt = f"'{review}', {description}"
    try:
        response = model.generate_content(prompt)
        return response.text.split('\n')[0].split(",")
    except Exception as e:
        return ["IR"]

In [10]:
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: colab 1.13.5 has a non-standard dependency specifier pytz>=2011n. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of colab or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [11]:
from tqdm.notebook import tqdm  # Import tqdm for displaying the progress bar
import pandas as pd

async def Augment_dataset(fileDir,filename):
    data = []
    data_df = pd.read_csv(f"{fileDir}/{filename}.csv",sep=",",header=0,quotechar='"')
    current_data = pd.read_csv(f"./Dataset/{filename}.csv",sep=",",header=0,quotechar='"')
    start_index = current_data.shape[0]
    # Calculate the total number of iterations
    total_reviews = len(data_df["review_text"]) - start_index

    # Initialize tqdm with the total number of iterations
    for review in tqdm(data_df["review_text"][start_index: ], total=total_reviews, desc="Classifying Reviews"):
        result = {}
        classification_list = await Classify(review)
        classification_list = [x for x in classification_list if x in classes]
        result["review_text"] = review
        for req in classes:
            if req in classification_list:
                result[req] = 1
            else:
                result[req] = 0
        data.append(result)
        # Create DataFrame from the data
        df = pd.DataFrame(data)
        final_df = pd.concat([current_data,df],axis=0)
        # Save DataFrame to a CSV file
        final_df.to_csv(f"./Dataset/{filename}.csv", index=False) 

In [12]:
await Augment_dataset("../code/_1_data_collection/output","dataset2")

Classifying Reviews:   0%|          | 0/44000 [00:00<?, ?it/s]